In this notebook we are using OpenAI GPT-4 for evaluation on MMLU dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
OPENAI_API_KEY = ""

In [3]:
from medplexity.benchmarks.mmlu.mmlu_dataset_builder import MMLUDatasetBuilder, MMLUSubsetConfig

dataset = MMLUDatasetBuilder().build_dataset(config_type=MMLUSubsetConfig.clinical_knowledge, split_type="validation")

In [4]:
example_data_point = next(dataset.__iter__())

In [5]:
example_data_point

MMLUDataPoint(input=MMLUInput(question='Mr Wood has just returned from surgery and has severe internal bleeding. Which of the following observations would you NOT expect to find on undertaking post-operative observations?', options=['Hypotension.', 'Bradycardia.', 'Confusion', 'Tachypnoea.']), expected_output='(B)', metadata=None)

In [6]:
from medplexity.benchmarks.mmlu.mmlu_prompt_template import MMLUPromptTemplate
from medplexity.benchmarks.mmlu.mmlu_dataset_builder import MMLUInput

def input_adapter(medqa_input: MMLUInput):
    prompt_template = MMLUPromptTemplate()

    return prompt_template.format(
        question=medqa_input.question,
        options=medqa_input.options
    )

In [7]:
print(input_adapter(example_data_point.input))

The following are multiple choice questions about medical knowledge. Solve them in a step-by-step fashion, starting by summarizing the available information. Output a JSON with the answer (give back only the letter) and an explanation for it.
Question: The energy for all forms of muscle contraction is provided by:
 (A) ATP (B) ADP (C) phosphocreatine (D) oxidative phosphorylation
Output: {"answer":"(A)","explanation":"The sole fuel for muscle contraction is adenosine triphosphate (ATP). During near maximal intense exercise the muscle store of ATP will be depleted in less than one second. Therefore, to maintain normal contractile function ATP must be continually resynthesized. These pathways include phosphocreatine and muscle glycogen breakdown, thus enabling substrate-level phosphorylation (‘anaerobic’) and oxidative phosphorylation by using reducing equivalents from carbohydrate and fat metabolism (‘aerobic’)."}


Question: Which of the following conditions does not show multifactoria

In [8]:
from medplexity.benchmarks.multiple_choice_utils import AnswerWithExplanation


def output_adapter(output_json: str) -> AnswerWithExplanation:
    parsed_output = AnswerWithExplanation.model_validate_json(output_json)

    return parsed_output

In [9]:
from medplexity.llms.openai_caller import OpenAI
from medplexity.chains.evaluation_adapter_chain import EvaluationAdapterChain

chain = EvaluationAdapterChain(
    llm=OpenAI(
        api_token=OPENAI_API_KEY
    ),
    input_adapter=input_adapter,
    output_adapter=output_adapter,
)

In [10]:
def comparator(expected_output: str, predicted_output: AnswerWithExplanation):

    return expected_output == predicted_output.answer

In [11]:
from medplexity.evaluators.sequential_evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(
    chain=chain,
    comparator=comparator
)

In [12]:
dataset[0].input

MMLUInput(question='Mr Wood has just returned from surgery and has severe internal bleeding. Which of the following observations would you NOT expect to find on undertaking post-operative observations?', options=['Hypotension.', 'Bradycardia.', 'Confusion', 'Tachypnoea.'])

In [13]:
evaluation = evaluator.evaluate(dataset[0:1])

100%|██████████| 1/1 [00:04<00:00,  4.99s/it]


In [14]:
evaluation.accuracy()

1.0

In [15]:
correct, incorrect = evaluation.partition_by_correctness()

In [16]:
correct

[EvaluationResult(input=MMLUInput(question='Mr Wood has just returned from surgery and has severe internal bleeding. Which of the following observations would you NOT expect to find on undertaking post-operative observations?', options=['Hypotension.', 'Bradycardia.', 'Confusion', 'Tachypnoea.']), input_metadata=None, expected_output='(B)', output=AnswerWithExplanation(answer='(B)', explanation='Severe internal bleeding is likely to cause hypotension (low blood pressure), tachycardia (rapid heart rate), and tachypnea (rapid breathing) as compensatory mechanisms to maintain perfusion. Confusion may also be present due to decreased cerebral perfusion. Bradycardia (slow heart rate) would not be expected in this situation, as the body would try to increase heart rate to compensate for the decreased blood volume.'), correct=True)]

In [17]:
incorrect

[]